<table class="table table-bordered">
    <tr>
        <th style="text-align:center; width:35%"><img src='https://dl.dropbox.com/s/qtzukmzqavebjd2/icon_smu.jpg' style="width: 300px; height: 90px; "></th>
    <th style="text-align:center;"><font size="4"> <br/>IS.215 - Practical 2 Recommenders in Python</font></th>
    </tr>
</table> 

### Content-based recommender system for movies

This recommender is built using content from IMDB top 250 English movies or https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7. The metadata used includes movie director, main actors and plot. 

Make sure Rake (Rapid Automatic Keyword Extraction algorithm) library is installed. If not, it can be installed via "pip install rake_nltk". Refer to https://pypi.org/project/rake-nltk/ for more information.

This python script is adapted from https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243

In [1]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

**Step 1: Read in and analyse input data**

In [2]:
df = pd.read_csv('IMDB_Top250Engmovies2_OMDB_Detailed.csv')
#df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')
df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",...,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [3]:
df.shape

(250, 38)

Use the following input features to base the recommendations.

In [4]:
df = df[['Title', 'Director', 'Actors', 'Plot', 'Genre']] # we only use these few features
df.head()

,Title,Director,Actors,Plot,Genre
0,The Shawshank Redemption,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,"Crime, Drama"
1,The Godfather,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"Crime, Drama"
2,The Godfather: Part II,Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"Crime, Drama"
3,The Dark Knight,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"Action, Crime, Drama"
4,12 Angry Men,Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,"Crime, Drama"


In [5]:
df.shape

(250, 5)

**Step 2a: Data pre-processing** Transforming the full names of actors and directors in single words so they are considered as unique values.

In [6]:
# discarding the commas between the actors' full names and getting only the first three names
df['Actors'] = df['Actors'].map(lambda x: x.split(',')[:3]) # to treat Actors as array, then pick the to 3

# putting the directors in a list of words
df['Director'] = df['Director'].map(lambda x: x.split(' ')) # split by whitespace, then store as array

df['Genre'] = df['Genre'].map(lambda x: x.split(',')) # split by comma as genre is given as CSV, then store as array

# merging first and last name for each actor and director into one word 
# to ensure no mix up between people sharing a first name
for index, row in df.iterrows():
    row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
    row['Director'] = ''.join(row['Director']).lower()
    row['Genre'] = [x.lower().replace(' ','') for x in row['Genre']]

**Step 2b: Data pre-processing on plot** Extracting the key words from the plot description.

In [7]:
# initializing the new column
df['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['Plot']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df.drop(columns = ['Plot'], inplace = True)
#if have error - use df.drop('Plot', axis=1, inplace=True)

In [8]:
# check the columns
df.head()

,Title,Director,Actors,Genre,Key_words
0,The Shawshank Redemption,frankdarabont,"[timrobbins, morganfreeman, bobgunton]","[crime, drama]","[eventual, redemption, common, decency, findin..."
1,The Godfather,francisfordcoppola,"[marlonbrando, alpacino, jamescaan]","[crime, drama]","[clandestine, empire, reluctant, son, organize..."
2,The Godfather: Part II,francisfordcoppola,"[alpacino, robertduvall, dianekeaton]","[crime, drama]","[portrayed, michael, tightens, grip, early, li..."
3,The Dark Knight,christophernolan,"[christianbale, heathledger, aaroneckhart]","[action, crime, drama]","[ability, joker, emerges, dark, knight, must, ..."
4,12 Angry Men,sidneylumet,"[martinbalsam, johnfiedler, leej.cobb]","[crime, drama]","[justice, forcing, reconsider, colleagues, evi..."


**Step 3: Create word representation - via bag of words using the values from the columns**

In [9]:
df['bag_of_words'] = ''
#Title should be omitted from bag of words creation
columns = df.columns[1:]
#print(columns)

for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Director':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df = df[['Title','bag_of_words']]

In [10]:
df.head()

,Title,bag_of_words
0,The Shawshank Redemption,frankdarabont timrobbins morganfreeman bobgunt...
1,The Godfather,francisfordcoppola marlonbrando alpacino james...
2,The Godfather: Part II,francisfordcoppola alpacino robertduvall diane...
3,The Dark Knight,christophernolan christianbale heathledger aar...
4,12 Angry Men,sidneylumet martinbalsam johnfiedler leej.cobb...


**Step 4: Create the model using count metrics**

In [11]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list that can be used to match the indexes
indices = pd.Series(df['Title'])
indices[:5]

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

In [12]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

[[1.         0.15789474 0.13764944 ... 0.05263158 0.05263158 0.05564149]
 [0.15789474 1.         0.36706517 ... 0.05263158 0.05263158 0.05564149]
 [0.13764944 0.36706517 1.         ... 0.04588315 0.04588315 0.04850713]
 ...
 [0.05263158 0.05263158 0.04588315 ... 1.         0.05263158 0.05564149]
 [0.05263158 0.05263158 0.04588315 ... 0.05263158 1.         0.05564149]
 [0.05564149 0.05564149 0.04850713 ... 0.05564149 0.05564149 1.        ]]


**Step 5: Test and run the model (recommender)**

In [13]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # getting the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index) # why don't return index 0? because it's the movie itself!
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df['Title'])[i])
        
    return recommended_movies

In [14]:
recommendations('The Dark Knight')

['The Dark Knight Rises',
 'Batman Begins',
 'The Prestige',
 'The Green Mile',
 'Witness for the Prosecution',
 'Out of the Past',
 'Rush',
 'The Godfather',
 'V for Vendetta',
 'Reservoir Dogs']